In [1]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import pandas_ta as ta
import numpy as np
import requests
from bs4 import BeautifulSoup

# Function to fetch S&P 500 tickers from Wikipedia
# def get_sp500_tickers():
#     url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
#     response = requests.get(url)
#     soup = BeautifulSoup(response.text, 'html.parser')
#     table = soup.find('table', {'id': 'constituents'})
#     tickers = [row.find('td').text.strip() for row in table.find_all('tr')[1:]]
#     return tickers

# # Get the S&P 500 tickers
# sp500_tickers = get_sp500_tickers()

# # Save the tickers to a CSV file
# pd.DataFrame(sp500_tickers, columns=["Ticker"]).to_csv('sp500_tickers.csv', index=False)


In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Function to fetch S&P 500 tickers from Wikipedia
def get_sp500_tickers():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})
    tickers = [row.find('td').text.strip() for row in table.find_all('tr')[1:]]
    return tickers[:10]# Get only the first 10 tickers

# Get the S&P 500 tickers
sp500_tickers = get_sp500_tickers()

# Display the tickers
print("S&P 500 Tickers:")
print(sp500_tickers)

# Save the tickers to a CSV file
pd.DataFrame(sp500_tickers, columns=["Ticker"]).to_csv('sp500_tickers.csv', index=False)


S&P 500 Tickers:
['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']


In [ ]:

# Define the timeframes and their corresponding date ranges

# Read the tickers from the CSV file
tickers_df = pd.read_csv('sp500_tickers.csv')
tickers = tickers_df['Ticker'].tolist()

timeframes = {
    # '15m': timedelta(days=60),
    # '1h': timedelta(days=730),
    '1d': timedelta(days=180),
    # '1wk': timedelta(days=3*365)
}

# Define the recent period (in days)
recent_period = 1

# Function to download data for a given ticker and timeframe
def download_data(ticker, timeframe, start_date, end_date):
    return yf.download(ticker, start=start_date, end=end_date, interval=timeframe)

# Dictionary to store data for each timeframe
data = {tf: {} for tf in timeframes}

# Download data for each ticker and timeframe
end_date = datetime.now()
for ticker in tickers:
    for tf, delta in timeframes.items():
        start_date = end_date - delta
        try:
            data[tf][ticker] = download_data(ticker, tf, start_date, end_date)
        except Exception as e:
            print(f"Failed to download data for {ticker} with timeframe {tf}: {e}")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [8]:


# List to store screener results
screener_results = []

# Applying the custom indicator and generating buy/sell signals
for tf in timeframes:
    for ticker in data[tf]:
        df = data[tf][ticker]
        
        # Round OHLC values to two decimal points
        df['Open'] = df['Open'].round(2)
        df['High'] = df['High'].round(2)
        df['Low'] = df['Low'].round(2)
        df['Close'] = df['Close'].round(2)
        
       # --- Existing calculations ---
        # Linear Regression Curves
        df['reg1'] = ta.linreg(df['Close'], length=10)
        df['reg2'] = ta.linreg(df['Close'], length=14)
        df['reg3'] = ta.linreg(df['Close'], length=30)

        # R-squared Calculation
        r2_length = 14
        df['r2_raw'] = df['Close'].rolling(window=r2_length).apply(
            lambda x: np.corrcoef(x, np.arange(r2_length))[0, 1]**2
        )
        df['r2'] = df['r2_raw'] * 100  # Normalized to [0, 100]
        df['r2_smoothed'] = df['r2'].rolling(window=3).mean()

        # --- Flatten the columns if they are a MultiIndex ---
        if isinstance(df.columns, pd.MultiIndex):
            # This will take the first level of the MultiIndex as the new columns.
            df.columns = df.columns.get_level_values(0)

        # --- RSI Calculation using the DataFrame accessor ---
        # Explicitly specify the 'Close' column.
        df.ta.rsi(close='Close', length=14, append=True)
        # This will add a new column named 'RSI_14' to df.

        # --- Buy and Sell Signals ---
        df['buy_signal'] = np.where((df['r2_smoothed'] > 90) & (df['RSI_14'] < 30), 1, 0)
        df['sell_signal'] = np.where((df['r2_smoothed'] > 90) & (df['RSI_14'] > 70), 1, 0)


                
                # Save the DataFrame to a CSV file
        df.to_csv(f'{ticker}_{tf}_data.csv')
        
        
        # Filter for recent periods
        recent_date_cutoff = df.index.max() - pd.Timedelta(days=recent_period)
        df = df[df.index >= recent_date_cutoff]
        
        # Filter rows with buy or sell signals
        df_filtered = df[(df['buy_signal'] == 1) | (df['sell_signal'] == 1)]
        
        # Append results to screener list
        if not df_filtered.empty:
            for index, row in df_filtered.iterrows():
                screener_results.append({
                    'Ticker': ticker,
                    'Date': index,
                    'Buy Signal': row['buy_signal'],
                    'Sell Signal': row['sell_signal']
                })

# Save screener results to a CSV file
screener_df = pd.DataFrame(screener_results)
# screener_df.to_csv('screener_results.csv', index=False)

# Display a sample of the screener results
print("Screener results:")
print(screener_df)


Screener results:
  Ticker       Date  Buy Signal  Sell Signal
0    MMM 2025-01-30         0.0          1.0
1    MMM 2025-01-31         0.0          1.0


# regession line crossed ta alerts

In [4]:

# Read the tickers from the CSV file
tickers_df = pd.read_csv('sp500_tickers.csv')
tickers = tickers_df['Ticker'].tolist()

# Define the timeframes and their corresponding date ranges
timeframes = {
    # '15m': timedelta(days=60),
    '1h': timedelta(days=30),
    # '1d': timedelta(days=3*365),
    # '1wk': timedelta(days=3*365)
}

# Define the recent period (in days)
recent_period = 1

# Function to download data for a given ticker and timeframe
def download_data(ticker, timeframe, start_date, end_date):
    return yf.download(ticker, start=start_date, end=end_date, interval=timeframe)

# Dictionary to store data for each timeframe
data = {tf: {} for tf in timeframes}

# Download data for each ticker and timeframe
end_date = datetime.now()
for ticker in tickers:
    for tf, delta in timeframes.items():
        start_date = end_date - delta
        try:
            data[tf][ticker] = download_data(ticker, tf, start_date, end_date)
        except Exception as e:
            print(f"Failed to download data for {ticker} with timeframe {tf}: {e}")


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [5]:

# List to store screener results
screener_results = []

# Applying the custom indicator and generating buy/sell signals
for tf in timeframes:
    for ticker in data[tf]:
        df = data[tf][ticker]
        
                # If the DataFrame columns are a MultiIndex, flatten them
        if isinstance(df.columns, pd.MultiIndex):
            df.columns = df.columns.get_level_values(0)

        
        # Round OHLC values to two decimal points
        df['Open'] = df['Open'].round(2)
        df['High'] = df['High'].round(2)
        df['Low'] = df['Low'].round(2)
        df['Close'] = df['Close'].round(2)
        
       # --- Existing calculations ---
        # Linear Regression Curves
        
        df['reg1'] = ta.linreg(df['Close'], length=25)
        df['reg2'] = ta.linreg(df['Close'], length=50)
        
        
        # df['reg3'] = ta.linreg(df['Close'], length=30)

        # R-squared Calculation
        # r2_length = 14
        # df['r2_raw'] = df['Close'].rolling(window=r2_length).apply(
        #     lambda x: np.corrcoef(x, np.arange(r2_length))[0, 1]**2
        # )
        # df['r2'] = df['r2_raw'] * 100  # Normalized to [0, 100]
        # df['r2_smoothed'] = df['r2'].rolling(window=3).mean()

        # --- Flatten the columns if they are a MultiIndex ---
        # if isinstance(df.columns, pd.MultiIndex):
        #     # This will take the first level of the MultiIndex as the new columns.
        #     df.columns = df.columns.get_level_values(0)

        # --- RSI Calculation using the DataFrame accessor ---
        # Explicitly specify the 'Close' column.
        # df.ta.rsi(close='Close', length=14, append=True)
        # This will add a new column named 'RSI_14' to df.

        # --- Buy and Sell Signals ---
        # df['buy_signal'] = np.where((df['r2_smoothed'] > 90) & (df['RSI_14'] < 30), 1, 0)
        # df['sell_signal'] = np.where((df['r2_smoothed'] > 90) & (df['RSI_14'] > 70), 1, 0)

        df['buy_signal'] = np.where(
            (df['reg1'] > df['reg2']) & (df['reg1'].shift(1) <= df['reg2'].shift(1)),
            1,
            0
        )
        
        # Sell Signal: When the 25-period line (reg1) crosses below the 50-period line (reg2)
        df['sell_signal'] = np.where(
            (df['reg1'] < df['reg2']) & (df['reg1'].shift(1) >= df['reg2'].shift(1)),
            1,
            0
        )
                
                # Save the DataFrame to a CSV file
        df.to_csv(f'{ticker}_{tf}_data.csv')
        
        
        # Filter for recent periods
        recent_date_cutoff = df.index.max() - pd.Timedelta(days=recent_period)
        df = df[df.index >= recent_date_cutoff]
        
        # Filter rows with buy or sell signals
        df_filtered = df[(df['buy_signal'] == 1) | (df['sell_signal'] == 1)]
        
        # Append results to screener list
        if not df_filtered.empty:
            for index, row in df_filtered.iterrows():
                screener_results.append({
                    'Ticker': ticker,
                    'Date': index,
                    'Buy Signal': row['buy_signal'],
                    'Sell Signal': row['sell_signal']
                })

# Save screener results to a CSV file
screener_df = pd.DataFrame(screener_results)
# screener_df.to_csv('screener_results.csv', index=False)

# Display a sample of the screener results
print("Screener results:")
print(screener_df)


Screener results:
  Ticker                      Date  Buy Signal  Sell Signal
0   ABBV 2025-01-31 16:30:00+00:00         1.0          0.0
1   ADBE 2025-01-31 15:30:00+00:00         0.0          1.0
